In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import *

In [2]:
df = pd.read_csv("./DATA/data.csv")

In [3]:
num_columns = len(df.columns)

In [4]:
df.head() #each row represents one second

,xx,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5


In [5]:
df = df.sample(frac=1) #shuffle dataset
data = df.drop(['y', 'xx'], axis=1).to_numpy()
labels = df['y'].to_numpy()

#map labels to binary - class 1 has epilepsy only. so class 1 for yes epilepsy, 0 for no
for (i,l) in enumerate(labels):
    if l != 1:
        labels[i] = 0
all_labels = list(set(df['y'].unique().tolist()))
print(all_labels)

[0, 1]


In [6]:
norm = np.linalg.norm(data)
data = data/norm

In [7]:
assert(len(data) == len(labels))
train, validate, test = np.split(data, [int(.6*len(data)), int(.8*len(data))])
train_labels, validate_labels, test_labels = np.split(labels, [int(.6*len(labels)), int(.8*len(labels))])

In [8]:
train.shape

(6900, 178)

In [9]:
EPOCHS = 10
BATCH_SIZE = 64
TIMESTEPS = 1
INPUT_SHAPE = (TIMESTEPS, num_columns-2)
DROPOUT = 0.2
UNITS = 256
ACTIVATION = 'relu'

In [10]:
def Model():
    input = layers.Input(shape=INPUT_SHAPE)
    print(input.shape)
    cell = layers.LSTMCell(32)
    rnn = layers.RNN(cell)(input)
    fc1 = layers.Dense(64, activation='relu')(rnn)
    output = layers.Dense(1)(fc1)

    model = tf.keras.Model(inputs=input, outputs=output)
    return model

In [11]:
model = Model()
model.summary()

(None, 1, 178)
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 178)]          0         
_________________________________________________________________
rnn (RNN)                    (None, 32)                27008     
_________________________________________________________________
dense (Dense)                (None, 64)                2112      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 29,185
Trainable params: 29,185
Non-trainable params: 0
_________________________________________________________________


In [12]:
def Reshape(input):
    size, features = input.shape
    return input.reshape(size, TIMESTEPS, features)

In [13]:
train, validate, test = Reshape(train), Reshape(validate), Reshape(test)

In [14]:
train.shape

(6900, 1, 178)

In [15]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
history = model.fit(train, train_labels, 
epochs=EPOCHS, 
validation_data=(validate, validate_labels), 
batch_size=BATCH_SIZE, 
verbose=2, 
shuffle=True)

Epoch 1/10
108/108 - 1s - loss: 0.5767 - accuracy: 0.8022 - val_loss: 0.5060 - val_accuracy: 0.7965
Epoch 2/10
108/108 - 0s - loss: 0.4974 - accuracy: 0.8022 - val_loss: 0.5053 - val_accuracy: 0.7965
Epoch 3/10
108/108 - 0s - loss: 0.4975 - accuracy: 0.8022 - val_loss: 0.5055 - val_accuracy: 0.7965
Epoch 4/10
108/108 - 0s - loss: 0.4972 - accuracy: 0.8022 - val_loss: 0.5054 - val_accuracy: 0.7965
Epoch 5/10
108/108 - 0s - loss: 0.4974 - accuracy: 0.8022 - val_loss: 0.5052 - val_accuracy: 0.7965
Epoch 6/10
108/108 - 0s - loss: 0.4970 - accuracy: 0.8022 - val_loss: 0.5050 - val_accuracy: 0.7965
Epoch 7/10
108/108 - 0s - loss: 0.4970 - accuracy: 0.8022 - val_loss: 0.5049 - val_accuracy: 0.7965
Epoch 8/10
108/108 - 0s - loss: 0.4967 - accuracy: 0.8022 - val_loss: 0.5049 - val_accuracy: 0.7965
Epoch 9/10
108/108 - 0s - loss: 0.4969 - accuracy: 0.8022 - val_loss: 0.5052 - val_accuracy: 0.7965
Epoch 10/10
108/108 - 0s - loss: 0.4967 - accuracy: 0.8022 - val_loss: 0.5050 - val_accuracy: 0.7965

In [17]:
# model.save("./Models/LSTM1.h5")

In [18]:
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')

In [19]:
# test_loss, test_acc = model.evaluate(test_data,  test_labels, verbose=2)

In [20]:
# prediction = model.predict(np.expand_dims(test_data[0], axis=0))
# class_labels.int2str(np.argmax(prediction))